`Install and Importing`

In [ ]:
!pip install osmnx folium geopy

In [ ]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import requests
from typing import List, Dict, Tuple

# Configure OSMnx
ox.settings.use_cache = True
ox.settings.log_console = True

In [ ]:
data = {
    "hospital": {
        "name": "Central Hospital",
        "latitude": 29.99512653425452,
        "longitude": 31.68462840171934,
        "type": "hospital"
    },
    "patients": [
        {"id": "DT", "name": "Patient DT", "latitude": 30.000417586266437, "longitude": 31.73960813272627},
        {"id": "GR", "name": "Patient GR", "latitude": 30.011344405285193, "longitude": 31.747827362371993},
        {"id": "R2", "name": "Patient R2", "latitude": 30.030388325206854, "longitude": 31.669231198639675},
        {"id": "R3_2", "name": "Patient R3_2", "latitude": 30.030940768851426, "longitude": 31.688371339937028},
        {"id": "IT", "name": "Patient IT", "latitude": 30.01285635906825, "longitude": 31.693811715848444}
    ]
}

hospital = {"latitude": data["hospital"]["latitude"], "longitude": data["hospital"]["longitude"]}

patients = {p["id"]: {"latitude": p["latitude"], "longitude": p["longitude"]} for p in data["patients"]}

In [ ]:
# function to get best route between two coordinates using OSRM
def best_route_between_points(lat1, lon1, lat2, lon2, return_geometry=False):
    """
    Calculate best driving route using OSRM.
    """
    coords_str = f"{lon1},{lat1};{lon2},{lat2}"
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            route = data['routes'][0]
            distance_km = route['distance'] / 1000
            duration_min = route['duration'] / 60
            geometry = route['geometry']['coordinates']

            if return_geometry:
                return distance_km, duration_min, geometry
            return distance_km, duration_min
        else:
            print("OSRM error: No route found")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None


In [ ]:
# function to compute the best full route starting/ending at the hospital
def best_full_route(hospital, patient_order, return_geometry=True):
    """
    Compute the best full route starting/ending at the hospital.
    """
    total_distance = 0.0
    total_duration = 0.0
    full_geometry = []

    stops = [hospital] + patient_order + [hospital]

    for i in range(len(stops)-1):
        lat1, lon1 = stops[i]["latitude"], stops[i]["longitude"]
        lat2, lon2 = stops[i+1]["latitude"], stops[i+1]["longitude"]

        result = best_route_between_points(lat1, lon1, lat2, lon2, return_geometry=True)
        if not result:
            continue
        dist, dur, geometry = result

        total_distance += dist
        total_duration += dur

        if i == 0:
            full_geometry.extend(geometry)
        else:
            full_geometry.extend(geometry[1:])

    if return_geometry:
        return total_distance, total_duration, full_geometry
    return total_distance, total_duration

In [ ]:
# function to visualize the full route on a folium map
def show_full_route_on_map(hospital, patient_order, filename):
    """
    Show and save full trip route (hospital -> patients -> hospital).
    """
    result = best_full_route(hospital, patient_order, return_geometry=True)
    if not result:
        return None
    total_distance, total_duration, geometry = result

    m = folium.Map(location=[hospital["latitude"], hospital["longitude"]], zoom_start=13)

    folium.Marker(
        [hospital["latitude"], hospital["longitude"]],
        popup="Hospital",
        icon=folium.Icon(color="red", icon="plus-sign")
    ).add_to(m)

    for idx, patient in enumerate(patient_order, 1):
        folium.Marker(
            [patient["latitude"], patient["longitude"]],
            popup=f"Patient {idx}",
            icon=folium.Icon(color="blue", icon="user")
        ).add_to(m)

    folium.PolyLine([(lat, lon) for lon, lat in geometry], color="green", weight=5, opacity=0.8).add_to(m)

    print(f"Total distance: {total_distance:.2f} km | Duration: {total_duration:.2f} min")
    m.save(filename)
    return m

In [ ]:
maps = {}
for pid, patient in patients.items():
    print(f"\n=== Full Route Between Hospital → {pid} ===")
    m = show_full_route_on_map(hospital, [patient], filename=f"full_route_hospital_{pid}.html")
    maps[pid] = m


=== Full Route Between Hospital → DT ===
Total distance: 22.82 km | Duration: 32.19 min

=== Full Route Between Hospital → GR ===
Total distance: 29.28 km | Duration: 35.70 min

=== Full Route Between Hospital → R2 ===
Total distance: 21.31 km | Duration: 32.10 min

=== Full Route Between Hospital → R3_2 ===
Total distance: 18.20 km | Duration: 27.54 min

=== Full Route Between Hospital → IT ===
Total distance: 18.94 km | Duration: 27.67 min


In [9]:
# Example: show Patient DT map
maps["DT"]

In [10]:
# Example: show Patient GR map
maps["GR"]

In [11]:
# Example: show Patient R2 map
maps["R2"]

In [12]:
# Example: show Patient R3_2 map
maps["R3_2"]

In [13]:
# Example: show Patient IT map
maps["IT"]